In [1]:
!git clone https://github.com/mritunjaypandey2k24/financial-analyst-rag.git
%cd financial-analyst-rag

Cloning into 'financial-analyst-rag'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 108 (delta 53), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (108/108), 35.17 KiB | 5.86 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/content/financial-analyst-rag


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [3]:
import torch
print(torch.cuda.is_available())  # Should return True

True


In [4]:

!ls

agent_pipeline.py  data  models  notebooks  preprocessing.py  README.md  utils


In [5]:
!pip install \
    sentence-transformers \
    faiss-cpu \
    numpy \
    transformers \
    beautifulsoup4 \
    pandas \
    PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 23.4 MB/s eta 0:00:00


In [6]:
# Install the CUDA-specific version for Colab
!pip install faiss-gpu-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MB 11.1 MB/s eta 0:00:00


In [7]:
from utils.retriever import DocumentRetriever
print("Retriever loaded successfully!")

Retriever loaded successfully!


In [8]:
from utils.data_fetcher import fetch_sec_filings

# Define companies and their CIKs
companies = {
    "Apple": "0000320193",
    "Microsoft": "0000789019"
}

# Fetch filings
for company, cik in companies.items():
    print(f"Fetching filings for {company}...")
    fetch_sec_filings(
        ticker=cik,
        filing_type="10-K",       # Annual reports
        save_directory=f"data/sec_filings/{company}",
        num_filings=3             # Number of filings to fetch
    )

Fetching filings for Apple...
Requesting: https://data.sec.gov/submissions/CIK0000320193.json
Downloading: https://www.sec.gov/Archives/edgar/data/320193/000032019325000079/aapl-20250927.htm
Downloading: https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm
Downloading: https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm
Done! Downloaded 3 filings for 0000320193.
Fetching filings for Microsoft...
Requesting: https://data.sec.gov/submissions/CIK0000789019.json
Downloading: https://www.sec.gov/Archives/edgar/data/789019/000095017025100235/msft-20250630.htm
Downloading: https://www.sec.gov/Archives/edgar/data/789019/000095017024087843/msft-20240630.htm
Downloading: https://www.sec.gov/Archives/edgar/data/789019/000095017023035122/msft-20230630.htm
Done! Downloaded 3 filings for 0000789019.


In [10]:
from preprocessing import preprocess_file
import os

# Define company folders
company_folders = ["data/sec_filings/Apple", "data/sec_filings/Microsoft"]
processed_documents = {}

# Preprocess each company's filings
for folder in company_folders:
    print(f"Processing filings in {folder}...")
    documents = []
    for file_name in os.listdir(folder):
        try:
            file_path = os.path.join(folder, file_name)
            cleaned_text = preprocess_file(file_path)  # Unified preprocessing
            documents.append(cleaned_text)
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
    processed_documents[folder] = documents

Processing filings in data/sec_filings/Apple...
Processing filings in data/sec_filings/Microsoft...


In [11]:
!pip install faiss-cpu

In [12]:
!nvidia-smi

Wed Jan 14 12:17:01 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
import faiss
print("FAISS installed and ready to use!")

FAISS installed and ready to use!


In [14]:
from utils.retriever import DocumentRetriever

indices = {}

for company, docs in processed_documents.items():
    # Initialize a retriever for each company
    retriever = DocumentRetriever()
    embeddings = retriever.create_embeddings(docs)
    retriever.build_index(embeddings)
    # Store the FAISS index
    indices[company] = retriever
    print(f"FAISS index built for {company}.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

FAISS index built for data/sec_filings/Apple.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

FAISS index built for data/sec_filings/Microsoft.


In [15]:
!pip install sentence-transformers faiss-cpu numpy

In [16]:
torch.cuda.is_available()

True

In [17]:
from agent_pipeline import FinancialRAGAgent

rag_agent = FinancialRAGAgent()
question = "What was the total revenue in the last annual report?"

for company, retriever in indices.items():
    print(f"Querying {company}...")
    answer = rag_agent.answer_query(question, processed_documents[company])
    print(f"Answer for {company}: {answer}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Answer for data/sec_filings/Apple: aapl-20230930 false 2023 FY 0000320193 P1Y 67 P1Y 25 P1Y 7 1 P412D P473D 0000320193 2022-09-25 2023-09-30 0000320193 us-gaap:CommonStockMember 2022-09-25 2023-09-30 0000320193 aapl:A1.375NotesDue2024Member 2022-09-25 2023-09-30 0000320193 aapl:A0.000Notesdue2025Member 2022-09-25 2023-09-30 0000320193 aapl:A0.875NotesDue2025Member 2022-09-25 2023-09-30 0000320193 aapl:A1.625NotesDue2026Member 2022-09-25 2023-09-30 0000320193 aapl:A2.000NotesDue2027Member 2022-09-25 2023-09-30 0000320193 aapl:A1.375NotesDue2029Member 2022-09-25 2023-09-30 0000320193 aapl:A3.050NotesDue2029Member 2022-09-25 2023-09-30 0000320193 aapl:A0.500Notesdue2031Member 2022-09-25 2023-09-30 0000320193 aapl:A3.600NotesDue2042Member 2022-09-25 2023-09-30 0000320193 2023-03-31 iso4217:USD 0000320193 2023-10-20 xbrli:shares 0000320193 us-gaap:ProductMember 2022-09-25 2023-09-30 0000320193 us-gaap:ProductMember 2021-09-26 2022- Question: What was the total revenue in the last annual rep

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer for data/sec_filings/Microsoft: 10-K --06-30 FY 0000789019 false P2Y P5Y P3Y P1Y false false 0000789019 msft:ShareRepurchaseProgramTwentyNineteenAndTwentyTwentyOneMember 2021-07-01 2022-06-30 0000789019 msft:IssuanceOfLongTermDebtTwoMember 2023-06-30 0000789019 msft:IssuanceOfLongTermDebtThreeMember 2024-06-30 0000789019 msft:IssuanceOfLongTermDebtNineMember srt:MinimumMember 2024-06-30 0000789019 msft:ShareRepurchaseProgramTwentyTwentyOneMember 2023-07-01 2023-09-30 0000789019 us-gaap:CashMember 2023-06-30 0000789019 us-gaap:NonoperatingIncomeExpenseMember us-gaap:AccumulatedGainLossNetCashFlowHedgeParentMember 2021-07-01 2022-06-30 0000789019 msft:AccumulatedTranslationAdjustmentAndOtherMember 2021-06-30 0000789019 msft:RegionalOperatingCentersMember 2023-07-01 2024-06-30 0000789019 msft:InflectionAiIncMember msft:ReprogrammedInterchangeLLCMember srt:MaximumMember 2024-06-30 0000789019 us-gaap:NonoperatingIncomeExpenseMember us-gaap:ForeignExchangeContractMember us-gaap:FairVa

In [18]:
!git add notebooks/financial_analyst_rag.ipynb
!git commit -m "Updated RAG pipeline notebook"
!git push origin main

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@3d1f15624100.(none)')
fatal: could not read Username for 'https://github.com': No such device or address
